In [1]:
import asyncio
import pandas as pd
import numpy as np
from datetime import datetime
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.config import settings
import json

In [2]:
test_data = {
  "access_token": None,
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

In [3]:
data = test_data
store_name = data.get('store_name')
seller_id = data.get('seller_id')
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_COMPETITORS_DETAILS
blob_shipping_cost = settings.BLOB_COMPETITORS_DETAILS(store_name)
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')
# Get dates to treat
list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)
print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')
df_processed_data = pd.DataFrame()
for date in list_dates_to_process:
    # Transform date to string
    date_to_process = date.strftime('%Y-%m-%d')
    print(f'Processing date: {date_to_process}')
    # Get blob with the date
    blob_prefix = blob_shipping_cost + f'date={date_to_process}/'
    # List all the files
    blobs = storage.list_blobs(bucket_name, blob_prefix)
    # Processing each blob
    for blob in blobs:
        print(f"Reading file: {blob.name}")
        content = storage.download_json(bucket_name, blob.name)

** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
*** Starting to process dates: 1 dates to process  ***
Processing date: 2024-09-20
Reading file: hubsmarthome/meli/api_response/item_competitors_details/date=2024-09-20/batch_0__process_time=2024-09-20T12:22:22.658330-03:00.json
Reading file: hubsmarthome/meli/api_response/item_competitors_details/date=2024-09-20/batch_1__process_time=2024-09-20T12:23:23.953020-03:00.json
Reading file: hubsmarthome/meli/api_response/item_competitors_details/date=2024-09-20/batch_2__process_time=2024-09-20T12:23:23.222408-03:00.json
Reading file: hubsmarthome/meli/api_response/item_competitors_details/date=2024-09-20/batch_3__process_time=2024-09-20T12:23:23.708533-03:00.json
Reading file: hubsmarthome/meli/api_response/item_competitors_d

In [15]:
json = content[0]

dict_content = {
                        'item_id' : json.get('item_id'),
                        'price_id': json.get('site_id'),
                        'competitors_type': 'input',
                        'category_id': json.get('category_id'),
                        'official_store_id': json.get('official_store_id'),
                        'initial_quantity': json.get('initial_quantity'),
                        'competitor_seller_id': json.get('seller_id'),
                        'buying_mode': json.get('buying_mode'),
                        'listing_type_id': json.get('listing_type_id'),
                        'condition': json.get('condition'),
                        'permalink': json.get('permalink'),
                        'status' : json.get('status'),
                        }


dict_content      

{'item_id': 'MLB3651481701',
 'price_id': 'MLB',
 'competitors_type': 'input',
 'category_id': 'MLB189908',
 'official_store_id': None,
 'initial_quantity': 21,
 'competitor_seller_id': 1207642281,
 'buying_mode': 'buy_it_now',
 'listing_type_id': 'gold_special',
 'condition': 'new',
 'permalink': 'https://produto.mercadolivre.com.br/MLB-3651481701-controlador-inteligente-fita-led-cct-wi-fible-alexa-google-_JM',
 'status': 'active'}

In [5]:
content[0]

{'id': 'MLB3651481701',
 'site_id': 'MLB',
 'title': 'Controlador Inteligente Fita Led Cct Wi-fi+ble Alexa Google',
 'seller_id': 1207642281,
 'category_id': 'MLB189908',
 'official_store_id': None,
 'price': 77.17,
 'base_price': 77.17,
 'original_price': None,
 'currency_id': 'BRL',
 'initial_quantity': 21,
 'sale_terms': [{'id': 'WARRANTY_TIME',
   'name': 'Tempo de garantia',
   'value_id': None,
   'value_name': '12 meses',
   'value_struct': {'number': 12, 'unit': 'meses'},
   'values': [{'id': None,
     'name': '12 meses',
     'struct': {'number': 12, 'unit': 'meses'}}],
   'value_type': 'number_unit'},
  {'id': 'WARRANTY_TYPE',
   'name': 'Tipo de garantia',
   'value_id': '2230279',
   'value_name': 'Garantia de fábrica',
   'value_struct': None,
   'values': [{'id': '2230279',
     'name': 'Garantia de fábrica',
     'struct': None}],
   'value_type': 'list'}],
 'buying_mode': 'buy_it_now',
 'listing_type_id': 'gold_special',
 'condition': 'new',
 'permalink': 'https://prod

In [6]:
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
store_name = 'hubsmarthome'
blob_path = f'{store_name}/meli/api_response/orders/'


storage.blob_exists(bucket_name= 'glm-store', blob_name=blob_path)

Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json


False

In [139]:
test_data = {
  "access_token": None,
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

In [43]:
async def process_date(date, storage, bucket_name, blob_shipping_cost):
    try:
        print(f'Processing date: {date}')
        blob_prefix = blob_shipping_cost + f'date={date}/'

        # Use run_in_executor to call synchronous methods without blocking
        loop = asyncio.get_event_loop()
        blobs = await loop.run_in_executor(None, storage.list_blobs, bucket_name, blob_prefix)

        df_processed_data = pd.DataFrame()

        for blob in blobs:
            print(f"Reading file: {blob.name}")
            content = await loop.run_in_executor(None, storage.download_json, bucket_name, blob.name)

            for json_content in content:
                df_ = await process_orders(json_content['results'])
                df_processed_data = pd.concat([df_processed_data, df_], ignore_index=True)

        if not df_processed_data.empty:
            df_processed_data['processed_json'] = pd.to_datetime(date)
            df_processed_data['process_time'] = datetime.now()

        print(f'*** Finished treating all data for date {date}. {df_processed_data.shape[0]} sales ***')

        return df_processed_data
    except Exception as e:
        print(f'Error processing date {date}: {e}')
        return pd.DataFrame()

def process_orders(json_data):
    try:
        df_ = pd.DataFrame()

        for sale in json_data:
            structured_sale = {
                # Your structured_sale dictionary here
            }

            # Create a new DataFrame for the structured sale
            df_sale = pd.DataFrame([structured_sale])

            # Concatenate only if df_ is not empty
            if df_.empty:
                df_ = df_sale  # Assign the first entry directly if df_ is empty
            else:
                df_ = pd.concat([df_, df_sale], ignore_index=True)

        return df_
    except Exception as e:
        print(f'Error processing json: {e}')
        return pd.DataFrame()



In [24]:
data = test_data
store_name = data.get('store_name')
seller_id = data.get('seller_id')

print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_ORDERS
blob_shipping_cost = settings.BLOB_ORDERS(store_name)
# Get dates to treat
list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)
list_dates_to_process = [date.strftime('%Y-%m-%d') for date in list_dates_to_process]
print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')
# Use asyncio.gather to process dates asynchronously
tasks = [process_date(date, storage, bucket_name, blob_shipping_cost) for date in list_dates_to_process]
results = await asyncio.gather(*tasks)
# Combine all DataFrames
df_all_processed_data = pd.concat(results, ignore_index=True)
print(f'*** Finished processing all dates. Total sales: {df_all_processed_data.shape[0]} ***')

In [47]:
date = list_dates_to_process[3]
print(f'Processing date: {date}')
blob_prefix = blob_shipping_cost + f'date={date}/'
# Use run_in_executor to call synchronous methods without blocking
loop = asyncio.get_event_loop()
blobs = await loop.run_in_executor(None, storage.list_blobs, bucket_name, blob_prefix)
df_processed_data = pd.DataFrame()
for blob in blobs:
    print(f"Reading file: {blob.name}")
    content = await loop.run_in_executor(None, storage.download_json, bucket_name, blob.name)
    for json_content in content:
        df_ = process_orders(json_content['results'])
        df_processed_data = pd.concat([df_processed_data, df_], ignore_index=True)
if not df_processed_data.empty:
    df_processed_data['processed_json'] = pd.to_datetime(date)
    df_processed_data['process_time'] = datetime.now()
print(f'*** Finished treating all data for date {date}. {df_processed_data.shape[0]} sales ***')

df_processed_data

Processing date: 2024-04-27
Reading file: hubsmarthome/meli/api_response/orders/date=2024-04-27/total_sales=384__data=2024-04-27__processing-time=2024-09-17T17:42:42.561756-03:00.json
*** Finished treating all data for date 2024-04-27. 8 sales ***


""
0
1
2
3
4
5
6
7


In [52]:
df_ = pd.DataFrame()
for sale in json_content:
    structured_sale = {
        # Your structured_sale dictionary here
    }
    # Create a new DataFrame for the structured sale
    df_sale = pd.DataFrame([structured_sale])
    # Concatenate only if df_ is not empty
    if df_.empty:
        df_ = df_sale  # Assign the first entry directly if df_ is empty
    else:
        df_ = pd.concat([df_, df_sale], ignore_index=True)

structured_sale

{}

In [44]:
process_orders(json_content['results'])

""
0


# Fetch historical orders

In [26]:
import aiohttp
import asyncio
from datetime import datetime, timedelta
import pandas as pd
import json
import aiofiles
from concurrent.futures import ThreadPoolExecutor
from flask import escape
import requests
import gc  # Garbage collection to free memory
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import log_process, authenticate
from src.config import settings

In [27]:
timezone_offset = "-03:00"
limit = 50

def fetch_data(date, offset, access_token, seller_id):
    formatted_date_start = date.strftime(f"%Y-%m-%dT00:00:00.000{timezone_offset}")
    formatted_date_end = (date + timedelta(days=1)).strftime(f"%Y-%m-%dT00:00:00.000{timezone_offset}")
    params = {
        'limit': limit,
        'offset': offset,
        'order.date_created.from': formatted_date_start,
        'order.date_created.to': formatted_date_end
    }
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    url = settings.URL_ORDERS(seller_id)

    response = requests.get(url, params=params, headers=headers) 
    print(response.json)
    return response

In [28]:
data = test_data
client_id = data.get('client_id')
client_secret = data.get('client_secret')
store_name = data.get('store_name')
seller_id = data.get('seller_id')
access_token = data.get('access_token')
print('** Defining authentication... **')
# Authenticate (assuming this is now centralized in utils.py or a similar file)
if not access_token:
    access_token = authenticate(client_id, client_secret)  # You can add this to a common module
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_ORDERS
start_date = datetime.today() - timedelta(days=365)
end_date = datetime.today() - timedelta(days=1)
blob_name = settings.BLOB_ORDERS(store_name)

In [29]:
offset = 0
responses = []
sales = []
date = end_date
# while True:
response = fetch_data(date, offset, access_token, seller_id)

response.json()

In [30]:
response

In [55]:
# Provided dictionary keys from the code
code_keys = [
    'reason', 'status_code', 'total_paid_amount', 'operation_type', 'transaction_amount', 
    'transaction_amount_refunded', 'date_approved', 'collector_id', 'coupon_id', 'installments',
    'authorization_code', 'taxes_amount', 'payment_id', 'date_last_modified', 'coupon_amount', 
    'payment_shipping_cost', 'installment_amount', 'activation_uri', 'overpaid_amount', 'card_id', 
    'payment_status_detail', 'issuer_id', 'payment_method_id', 'payment_type', 'deferred_period', 
    'atm_transfer_reference_transaction_id', 'atm_transfer_reference_company_id', 'site_id', 
    'payer_id', 'order_id', 'currency_id', 'payment_status', 'shipping_id', 'fulfilled', 'seller_id', 
    'buyer_id', 'item_id', 'item_title', 'item_category_id', 'item_variation_id', 'seller_custom_field', 
    'global_price', 'net_weight', 'warranty', 'condition', 'seller_sku', 'quantity', 'unit_price', 
    'full_unit_price', 'manufacturing_days', 'requested_quantity_measure', 'requested_quantity_value', 
    'sale_fee', 'listing_type_id', 'base_exchange_rate', 'base_currency_id', 'bundle', 'element_id', 
    'date_created', 'date_closed', 'status', 'expiration_date', 'date_last_updated', 'last_updated', 
    'comment', 'pack_id', 'coupon_amount', 'coupon_id', 'shipping_cost', 'pickup_id', 'status_detail', 
    'total_amount', 'paid_amount', 'context_application', 'context_product_id', 'context_channel', 
    'context_site'
]

# Keys from the schema
schema_keys = [
    'reason', 'status_code', 'total_paid_amount', 'operation_type', 'transaction_amount',
    'transaction_amount_refunded', 'date_approved', 'collector_id', 'coupon_id', 'installments',
    'authorization_code', 'taxes_amount', 'payment_id', 'date_last_modified', 'coupon_amount', 
    'shipping_cost', 'installment_amount', 'activation_uri', 'overpaid_amount', 'card_id', 
    'status_detail', 'issuer_id', 'payment_method_id', 'payment_type', 'deferred_period', 
    'atm_transfer_reference_transaction_id', 'atm_transfer_reference_company_id', 'site_id', 
    'payer_id', 'order_id', 'currency_id', 'payment_status', 'shipping_id', 'fulfilled', 'seller_id', 
    'buyer_id', 'item_id', 'item_title', 'item_category_id', 'item_variation_id', 'seller_custom_field', 
    'global_price', 'net_weight', 'warranty', 'condition', 'seller_sku', 'quantity', 'unit_price', 
    'full_unit_price', 'manufacturing_days', 'requested_quantity_measure', 'requested_quantity_value', 
    'sale_fee', 'listing_type_id', 'base_exchange_rate', 'base_currency_id', 'bundle', 'element_id', 
    'date_created', 'date_closed', 'status', 'expiration_date', 'date_last_updated', 'last_updated', 
    'comment', 'pack_id', 'pickup_id', 'total_amount', 'paid_amount', 'context_application', 
    'context_product_id', 'context_channel', 'context_site', 'processed_json', 'process_time'
]

# Find keys that are in the dictionary but not in the schema
extra_keys = set(code_keys) - set(schema_keys)
extra_keys = set(schema_keys) - set(code_keys)  

extra_keys

{'process_time', 'processed_json'}

In [58]:
extra_keys_ = set(code_keys) - set(schema_keys)
extra_keys = set(schema_keys) - set(code_keys)  
extra_keys

{'process_time', 'processed_json'}

In [59]:
extra_keys_

{'payment_shipping_cost', 'payment_status_detail'}

In [17]:
def authenticate(client_id, client_secret):
  url = "https://api.mercadolibre.com/oauth/token"
  payload = {
      'grant_type': 'client_credentials',
      'client_id': client_id,
      'client_secret': client_secret
  }
  response = requests.post(url, data=payload)
  if response.status_code == 200:
    return response.json()['access_token']
  else:
    raise Exception("Authentication failed")

In [29]:
import requests

client_id     = "4959083987776428"
client_secret = "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw"

access_token = authenticate(client_id, client_secret)

In [172]:
URL_SHIPPING_COST = f'https://api.mercadolibre.com/users/189643563/shipping_options/free?item_id={item_id}'
URL_SHIPPING_COST

In [197]:
item_id = 'MLB4237237960'
# url = f'https://api.mercadolibre.com/items/{item_id}/sale_price?context=channel_marketplace, channel_mshops'
url =  f'https://api.mercadolibre.com/users/189643563/shipping_options/free?item_id={item_id}&channel=mshops'

headers = {
        'Authorization': f'Bearer {access_token}'
    }
response = requests.get(url, headers=headers)
data = response.json()
data

{'coverage': {'all_country': {'list_cost': 21.45,
   'currency_id': 'BRL',
   'billable_weight': 750}}}

In [187]:
url = 'https://api.mercadolibre.com/sites/MLB/listing_prices?price=500&listing_type_id=gold_pro'
response = requests.get(url, headers=headers)
response.json()

[{'currency_id': 'BRL',
  'free_relist': False,
  'listing_exposure': 'highest',
  'listing_fee_amount': 0,
  'listing_fee_details': {'fixed_fee': 0, 'gross_amount': 0},
  'listing_type_id': 'gold_pro',
  'listing_type_name': 'Premium',
  'requires_picture': True,
  'sale_fee_amount': 80,
  'sale_fee_details': {'fixed_fee': 0,
   'gross_amount': 80,
   'percentage_fee': 16},
  'stop_time': '2044-09-15T00:00:00.000-04:00'}]

# Getting costs for mlshops

In [198]:
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime

In [199]:
data = test_data
client_id = data.get('client_id')
client_secret = data.get('client_secret')
store_name = data.get('store_name')
seller_id = data.get('seller_id')
access_token = data.get('access_token')
print('** Defining authentication... **')
# Authenticate (assuming this is now centralized in utils.py or a similar file)
if not access_token:
    access_token = authenticate(client_id, client_secret)  # You can add this to a common module
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_COSTS
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')

# Fetch item IDs from the storage bucket
blob_items_prefix = f'{store_name}/meli/api_response/item_detail/date={today_str}/'
blob_prices = settings.BLOB_PRICES(store_name)

items_id = fetch_items_from_storage(
storage, 
bucket_name, 
blob_items_prefix, 
key_names=['id','price', 'category_id', 'listing_type_id']
)

prices = fetch_items_from_storage(
storage, 
bucket_name, 
blob_prices, 
key_names=['id','price', 'category_id', 'listing_type_id']
)



** Defining authentication... **
** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-20/batch_id=0__total_items=100__process_time=2024-09-20T09:10:10.048471-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-20/batch_id=1__total_items=100__process_time=2024-09-20T09:10:10.942620-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-20/batch_id=2__total_items=100__process_time=2024-09-20T09:10:10.829410-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-20/batch_id=3__total_items=100__process_time=2024-09-20T09:10:10.931497-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-20/batch_id=4__

In [200]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.config import settings
import json

In [201]:
test_data = {
  "access_token": "APP_USR-2951712600123976-092103-9ed59662b4bd7dfa1e8eb085d50e677c-150350803",
  "client_id": "2951712600123976",
  "client_secret": "QprAIl8ydXzcxFVHjnIHT6fUQ8KpzADV",
  "seller_id": 150350803,
  "store_name": "berga eletro_pr"
}

In [202]:
data = test_data
store_name = data.get('store_name')
seller_id = data.get('seller_id')
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_PRICES
blob_shipping_cost = settings.BLOB_PRICES(store_name)
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')
# Get dates to treat
list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)
print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')
df_processed_data = pd.DataFrame()
for date in list_dates_to_process:
    # Transform date to string
    date_to_process = date.strftime('%Y-%m-%d')
    print(f'Processing date: {date_to_process}')
    # Get blob with the date
    blob_prefix = blob_shipping_cost + f'date={date_to_process}/'
    # List all the files
    blobs = storage.list_blobs(bucket_name, blob_prefix)

** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
*** Starting to process dates: 0 dates to process  ***


In [203]:
date_to_process = '2024-09-21'
print(f'Processing date: {date_to_process}')
# Get blob with the date
blob_prefix = blob_shipping_cost + f'date={date_to_process}/'
# List all the files
blobs = storage.list_blobs(bucket_name, blob_prefix)
content = storage.download_json(bucket_name, blob.name)

Processing date: 2024-09-21


In [247]:
def process_prices(json):

    try:
        extracted_data = []

        # Dicionário temporário para priorizar os preços por canal
        price_by_channel = {}

        for price in json['prices']:
            channel = price['conditions']['context_restrictions']
            if len(channel) == 1:
                channel = channel[0]


            # Se ainda não há preço para o canal ou se o preço atual é promoção, atualiza
            if channel not in price_by_channel or price['type'] == 'promotion':
                price_by_channel[channel] = {
                    'item_id': json.get('id'),
                    'price_id': price.get('id'),
                    'regular_amount': price.get('regular_amount'),
                    'price': price.get('amount'),
                    'channel': channel,
                    'last_updated': price.get('last_updated')
                }

        # Converte os valores armazenados para uma lista
        extracted_data.extend(price_by_channel.values())

        return extracted_data
    
    except:
        print(f'Error processing json: {json}')

In [249]:
process_prices(content[1])

Error processing json: {'id': 'MLB4509988752', 'prices': [{'id': '15', 'type': 'standard', 'amount': 134.82, 'regular_amount': None, 'currency_id': 'BRL', 'last_updated': '2024-06-26T14:09:50Z', 'conditions': {'context_restrictions': [], 'start_time': None, 'end_time': None}}], 'item_id': 'MLB4509988752'}


In [252]:

price

NameError: name 'price' is not defined

In [275]:
content[5]

{'id': 'MLB3757145693',
 'prices': [{'id': '26',
   'type': 'standard',
   'amount': 249.99,
   'regular_amount': None,
   'currency_id': 'BRL',
   'last_updated': '2024-09-09T12:00:20Z',
   'conditions': {'context_restrictions': [],
    'start_time': None,
    'end_time': None}},
  {'id': '25',
   'type': 'promotion',
   'amount': 169.99,
   'regular_amount': 199.99,
   'currency_id': 'BRL',
   'last_updated': '2024-09-07T03:32:33Z',
   'conditions': {'context_restrictions': ['channel_marketplace'],
    'start_time': '2024-09-01T03:00:00Z',
    'end_time': '2024-10-01T02:59:59Z'}}],
 'item_id': 'MLB3757145693'}

In [278]:
def process_prices(json):

    try:
        extracted_data = []
        # Dicionário temporário para priorizar os preços por canal
        price_by_channel = {}
        for price in json['prices']:
            channel = price['conditions']['context_restrictions']
            if len(channel) == 1:
                channel = channel[0]

                # Se ainda não há preço para o canal ou se o preço atual é promoção, atualiza
                if channel not in price_by_channel or price['type'] == 'promotion':
                    price_by_channel[channel] = {
                        'item_id': json.get('id'),
                        'price_id': price.get('id'),
                        'regular_amount': price.get('regular_amount'),
                        'price': price.get('amount'),
                        'channel': channel,
                        'last_updated': price.get('last_updated')
                    }
        # Converte os valores armazenados para uma lista
        extracted_data.extend(price_by_channel.values())

        return extracted_data
    
    except:
        print(f'Error processing json: {json}')
        

In [283]:
process_prices(content[8])

[{'item_id': 'MLB3788435711',
  'price_id': '10',
  'regular_amount': 399.0,
  'price': 239.4,
  'channel': 'channel_marketplace',
  'last_updated': '2024-09-07T04:59:44Z'},
 {'item_id': 'MLB3788435711',
  'price_id': '5',
  'regular_amount': None,
  'price': 199,
  'channel': 'channel_mshops',
  'last_updated': '2024-08-20T18:52:43Z'}]

In [282]:
content[6]

{'id': 'MLB3758665407',
 'prices': [{'id': '1',
   'type': 'standard',
   'amount': 159.1,
   'regular_amount': None,
   'currency_id': 'BRL',
   'last_updated': '2024-06-27T19:09:48Z',
   'conditions': {'context_restrictions': [],
    'start_time': None,
    'end_time': None}},
  {'id': '45',
   'type': 'promotion',
   'amount': 151.14,
   'regular_amount': 159.1,
   'currency_id': 'BRL',
   'last_updated': '2024-09-18T18:35:39Z',
   'conditions': {'context_restrictions': ['channel_marketplace'],
    'start_time': '2024-09-01T03:00:00Z',
    'end_time': '2024-10-01T02:59:59Z'}}],
 'item_id': 'MLB3758665407'}

In [345]:
df = pd.DataFrame({'a': ['aaaa_bbbb','aaaa_bbbb']})
df['a'].apply(lambda x : x.split('_')[-1])

0    bbbb
1    bbbb
Name: a, dtype: object

In [351]:
df['a'].apply(lambda x : x.replace('aaaa', ''))

0    _bbbb
1    _bbbb
Name: a, dtype: object

In [354]:
    bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)

    # Define paths and table names from the config
    bucket_name = settings.BUCKET_STORES
    table_management = settings.TABLE_MANAGEMENT
    destiny_table = settings.TABLE_COSTS

    # Define today's date
    today_str = datetime.today().strftime('%Y-%m-%d')

    # Getting params to see costs
    query = f'''
        with items_details as (
        select distinct
            item_id,
            listing_type,
            category_id
        from datalake-v2-424516.datalake_v2.items_details
        where
            1=1
            and date(correspondent_date) = current_date()
            and seller_id = {seller_id}
        )

        select 
        p.item_id,
        d.listing_type,
        d.category_id,
        p.price,
        p.channel
        from datalake-v2-424516.datalake_v2.items_prices p
        inner join items_details d
        on p.item_id = d.item_id
        where 
        1=1
        and date(p.correspondent_date) = current_date()
    '''

    df_params = bigquery.run_query(query)

Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json


[{'item_id': 'MLB3636253863',
  'listing_type': 'gold_special',
  'category_id': 'MLB455251',
  'price': 795.0,
  'channel': 'channel_mshops'},
 {'item_id': 'MLB3326149829',
  'listing_type': 'gold_special',
  'category_id': 'MLB7073',
  'price': 178.99,
  'channel': 'channel_mshops'},
 {'item_id': 'MLB3649200017',
  'listing_type': 'gold_special',
  'category_id': 'MLB7070',
  'price': 89.99,
  'channel': 'channel_mshops'},
 {'item_id': 'MLB2059268709',
  'listing_type': 'gold_special',
  'category_id': 'MLB269928',
  'price': 78.99,
  'channel': 'channel_mshops'},
 {'item_id': 'MLB3775717904',
  'listing_type': 'gold_special',
  'category_id': 'MLB269928',
  'price': 799.2,
  'channel': 'channel_mshops'},
 {'item_id': 'MLB3204753274',
  'listing_type': 'gold_special',
  'category_id': 'MLB269928',
  'price': 297.99,
  'channel': 'channel_mshops'},
 {'item_id': 'MLB3474827016',
  'listing_type': 'gold_special',
  'category_id': 'MLB269928',
  'price': 152.63,
  'channel': 'channel_mar

In [363]:
type(df_params.to_dict(orient='records'))

list

In [364]:
blob_items_prefix = f'{store_name}/meli/api_response/item_detail/date={today_str}/'
items_id = fetch_items_from_storage(
storage, 
bucket_name, 
blob_items_prefix, 
key_names=['id','price', 'category_id', 'listing_type_id']
)


Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-21/batch_id=0__total_items=100__process_time=2024-09-21T15:24:24.192832-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-21/batch_id=1__total_items=100__process_time=2024-09-21T15:24:24.540762-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-21/batch_id=2__total_items=100__process_time=2024-09-21T15:24:24.955516-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-21/batch_id=3__total_items=100__process_time=2024-09-21T15:24:24.977460-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-21/batch_id=4__total_items=100__process_time=2024-09-21T15:24:24.490120-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-21/batch_id=5__total_items=100__process_time=2024-09-21T15:24:24.709973-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-21/batch_id=

In [378]:
params = {"id": "MLB3844407705",
      "listing_type_id": "gold_special",
      "category_id": "MLB269928",
      "price": 139.64,
      "channel": "marketplace"}

In [379]:
url = 'https://api.mercadolibre.com/sites/MLB/listing_prices'

response = requests.get(url, headers=headers, params=params)
response.json()

{'currency_id': 'BRL',
 'free_relist': False,
 'listing_exposure': 'highest',
 'listing_fee_amount': 0,
 'listing_fee_details': {'fixed_fee': 0, 'gross_amount': 0},
 'listing_type_id': 'gold_special',
 'listing_type_name': 'Clássico',
 'requires_picture': True,
 'sale_fee_amount': 16.06,
 'sale_fee_details': {'fixed_fee': 0,
  'gross_amount': 16.06,
  'percentage_fee': 11.5},
 'stop_time': '2044-09-16T00:00:00.000-04:00'}